In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import scanpy.api as sc

import perturbseq as perturb
sc.logging.print_versions()


scanpy==1.5.1 anndata==0.7.1 umap==0.4.4 numpy==1.17.2 scipy==1.4.1 pandas==0.24.2 scikit-learn==0.22 statsmodels==0.11.0 python-igraph==0.7.1 louvain==0.6.1 leidenalg==0.8.1


In [55]:
PROTEIN='TP53'
DATA='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14'
SUBSAMPLED=DATA+'/'+PROTEIN+'/subsampled/A549.'+PROTEIN
SUBSAMPLED=DATA+'/'+PROTEIN+'/subsampled/A549.'+PROTEIN

counts=20000
pref=SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.1000cells.analysis'


FIGS=DATA+'/Figures_regressed_out_vbc/'
print(FIGS)

/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14/Figures_regressed_out_vbc/


In [56]:
!mkdir -p /ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14/Figures_regressed_out_vbc/


In [57]:
adata=sc.read(SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.regress_out_vbc.analysis.h5ad',cache=False)
adata

Only considering the two last: ['.analysis', '.h5ad'].
Only considering the two last: ['.analysis', '.h5ad'].


AnnData object with n_obs × n_vars = 162314 × 1224 
    obs: 'TP53_A159P', 'TP53_A159V', 'TP53_C135F', 'TP53_C135Y', 'TP53_C141Y', 'TP53_C176F', 'TP53_C176Y', 'TP53_C238F', 'TP53_C238Y', 'TP53_C242F', 'TP53_C275F', 'TP53_C275Y', 'TP53_C277F', 'TP53_E271K', 'TP53_E285K', 'TP53_E286K', 'TP53_G105C', 'TP53_G154V', 'TP53_G244C', 'TP53_G245C', 'TP53_G245D', 'TP53_G245S', 'TP53_G245V', 'TP53_G266E', 'TP53_G266R', 'TP53_G266V', 'TP53_H179R', 'TP53_H179Y', 'TP53_H193L', 'TP53_H193R', 'TP53_H193Y', 'TP53_H214R', 'TP53_I195T', 'TP53_K132N', 'TP53_K132R', 'TP53_L194R', 'TP53_M237I', 'TP53_M246V', 'TP53_P151S', 'TP53_P152L', 'TP53_P250L', 'TP53_P278L', 'TP53_P278S', 'TP53_R110L', 'TP53_R158H', 'TP53_R158L', 'TP53_R175H', 'TP53_R213Q', 'TP53_R248L', 'TP53_R248W', 'TP53_R249M', 'TP53_R249S', 'TP53_R273C', 'TP53_R273H', 'TP53_R273L', 'TP53_R280K', 'TP53_R280T', 'TP53_R282W', 'TP53_R337C', 'TP53_R337L', 'TP53_S127F', 'TP53_S241C', 'TP53_S241F', 'TP53_V157F', 'TP53_V173L', 'TP53_V173M', 'TP53_V216M', '

In [14]:
adata.obsp['connectivities']

<162314x162314 sparse matrix of type '<class 'numpy.float32'>'
	with 3916024 stored elements in Compressed Sparse Row format>

In [6]:
adata2=perturb.pp.subsample_cells(adata,1000,'mut.std')

In [7]:
cells=adata2.obs_names
print(cells[:5])

Index(['ACGCCGATCAGGATCT-1-0-0', 'ACGGAGATCCAGAAGG-1-0-0',
       'AGACGTTGTGTGCGTC-1-0-0', 'CAGCATAGTATCAGTC-1-0-0',
       'CGCCAAGTCATATCGG-1-0-0'],
      dtype='object')


In [10]:
del adata.obsp["connectivities"]

In [12]:
del adata.obsp["distances"]

In [14]:
adata3=adata[list(cells),:].copy()
adata3

AnnData object with n_obs × n_vars = 85778 × 1224 
    obs: 'TP53_A159P', 'TP53_A159V', 'TP53_C135F', 'TP53_C135Y', 'TP53_C141Y', 'TP53_C176F', 'TP53_C176Y', 'TP53_C238F', 'TP53_C238Y', 'TP53_C242F', 'TP53_C275F', 'TP53_C275Y', 'TP53_C277F', 'TP53_E271K', 'TP53_E285K', 'TP53_E286K', 'TP53_G105C', 'TP53_G154V', 'TP53_G244C', 'TP53_G245C', 'TP53_G245D', 'TP53_G245S', 'TP53_G245V', 'TP53_G266E', 'TP53_G266R', 'TP53_G266V', 'TP53_H179R', 'TP53_H179Y', 'TP53_H193L', 'TP53_H193R', 'TP53_H193Y', 'TP53_H214R', 'TP53_I195T', 'TP53_K132N', 'TP53_K132R', 'TP53_L194R', 'TP53_M237I', 'TP53_M246V', 'TP53_P151S', 'TP53_P152L', 'TP53_P250L', 'TP53_P278L', 'TP53_P278S', 'TP53_R110L', 'TP53_R158H', 'TP53_R158L', 'TP53_R175H', 'TP53_R213Q', 'TP53_R248L', 'TP53_R248W', 'TP53_R249M', 'TP53_R249S', 'TP53_R273C', 'TP53_R273H', 'TP53_R273L', 'TP53_R280K', 'TP53_R280T', 'TP53_R282W', 'TP53_R337C', 'TP53_R337L', 'TP53_S127F', 'TP53_S241C', 'TP53_S241F', 'TP53_V157F', 'TP53_V173L', 'TP53_V173M', 'TP53_V216M', 'T

In [15]:
adata=perturb.pp.subsample_cells(adata3,1000,'mut.std')
adata

View of AnnData object with n_obs × n_vars = 85778 × 1224 
    obs: 'TP53_A159P', 'TP53_A159V', 'TP53_C135F', 'TP53_C135Y', 'TP53_C141Y', 'TP53_C176F', 'TP53_C176Y', 'TP53_C238F', 'TP53_C238Y', 'TP53_C242F', 'TP53_C275F', 'TP53_C275Y', 'TP53_C277F', 'TP53_E271K', 'TP53_E285K', 'TP53_E286K', 'TP53_G105C', 'TP53_G154V', 'TP53_G244C', 'TP53_G245C', 'TP53_G245D', 'TP53_G245S', 'TP53_G245V', 'TP53_G266E', 'TP53_G266R', 'TP53_G266V', 'TP53_H179R', 'TP53_H179Y', 'TP53_H193L', 'TP53_H193R', 'TP53_H193Y', 'TP53_H214R', 'TP53_I195T', 'TP53_K132N', 'TP53_K132R', 'TP53_L194R', 'TP53_M237I', 'TP53_M246V', 'TP53_P151S', 'TP53_P152L', 'TP53_P250L', 'TP53_P278L', 'TP53_P278S', 'TP53_R110L', 'TP53_R158H', 'TP53_R158L', 'TP53_R175H', 'TP53_R213Q', 'TP53_R248L', 'TP53_R248W', 'TP53_R249M', 'TP53_R249S', 'TP53_R273C', 'TP53_R273H', 'TP53_R273L', 'TP53_R280K', 'TP53_R280T', 'TP53_R282W', 'TP53_R337C', 'TP53_R337L', 'TP53_S127F', 'TP53_S241C', 'TP53_S241F', 'TP53_V157F', 'TP53_V173L', 'TP53_V173M', 'TP53_V2

In [16]:
adata._init_as_actual(adata.copy())

In [17]:
adata

AnnData object with n_obs × n_vars = 85778 × 1224 
    obs: 'TP53_A159P', 'TP53_A159V', 'TP53_C135F', 'TP53_C135Y', 'TP53_C141Y', 'TP53_C176F', 'TP53_C176Y', 'TP53_C238F', 'TP53_C238Y', 'TP53_C242F', 'TP53_C275F', 'TP53_C275Y', 'TP53_C277F', 'TP53_E271K', 'TP53_E285K', 'TP53_E286K', 'TP53_G105C', 'TP53_G154V', 'TP53_G244C', 'TP53_G245C', 'TP53_G245D', 'TP53_G245S', 'TP53_G245V', 'TP53_G266E', 'TP53_G266R', 'TP53_G266V', 'TP53_H179R', 'TP53_H179Y', 'TP53_H193L', 'TP53_H193R', 'TP53_H193Y', 'TP53_H214R', 'TP53_I195T', 'TP53_K132N', 'TP53_K132R', 'TP53_L194R', 'TP53_M237I', 'TP53_M246V', 'TP53_P151S', 'TP53_P152L', 'TP53_P250L', 'TP53_P278L', 'TP53_P278S', 'TP53_R110L', 'TP53_R158H', 'TP53_R158L', 'TP53_R175H', 'TP53_R213Q', 'TP53_R248L', 'TP53_R248W', 'TP53_R249M', 'TP53_R249S', 'TP53_R273C', 'TP53_R273H', 'TP53_R273L', 'TP53_R280K', 'TP53_R280T', 'TP53_R282W', 'TP53_R337C', 'TP53_R337L', 'TP53_S127F', 'TP53_S241C', 'TP53_S241F', 'TP53_V157F', 'TP53_V173L', 'TP53_V173M', 'TP53_V216M', 'T

In [19]:
new_mutstd=[]
for i in range(len(adata.obs_names)):
    muthere=adata.obs['mut.std'][i]
    if muthere=='uninfected':
        muthere='unassigned'
    new_mutstd.append(muthere)
adata.obs['mut.std']=new_mutstd

In [20]:
adata.write(SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.regress_out_vbc.analysis.1000cells.h5ad')

... storing 'mut.std' as categorical


In [58]:
adata=sc.read(SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.regress_out_vbc.analysis.1000cells.h5ad',cache=False)
adata

Only considering the two last: ['.1000cells', '.h5ad'].
Only considering the two last: ['.1000cells', '.h5ad'].


AnnData object with n_obs × n_vars = 85778 × 1224 
    obs: 'TP53_A159P', 'TP53_A159V', 'TP53_C135F', 'TP53_C135Y', 'TP53_C141Y', 'TP53_C176F', 'TP53_C176Y', 'TP53_C238F', 'TP53_C238Y', 'TP53_C242F', 'TP53_C275F', 'TP53_C275Y', 'TP53_C277F', 'TP53_E271K', 'TP53_E285K', 'TP53_E286K', 'TP53_G105C', 'TP53_G154V', 'TP53_G244C', 'TP53_G245C', 'TP53_G245D', 'TP53_G245S', 'TP53_G245V', 'TP53_G266E', 'TP53_G266R', 'TP53_G266V', 'TP53_H179R', 'TP53_H179Y', 'TP53_H193L', 'TP53_H193R', 'TP53_H193Y', 'TP53_H214R', 'TP53_I195T', 'TP53_K132N', 'TP53_K132R', 'TP53_L194R', 'TP53_M237I', 'TP53_M246V', 'TP53_P151S', 'TP53_P152L', 'TP53_P250L', 'TP53_P278L', 'TP53_P278S', 'TP53_R110L', 'TP53_R158H', 'TP53_R158L', 'TP53_R175H', 'TP53_R213Q', 'TP53_R248L', 'TP53_R248W', 'TP53_R249M', 'TP53_R249S', 'TP53_R273C', 'TP53_R273H', 'TP53_R273L', 'TP53_R280K', 'TP53_R280T', 'TP53_R282W', 'TP53_R337C', 'TP53_R337L', 'TP53_S127F', 'TP53_S241C', 'TP53_S241F', 'TP53_V157F', 'TP53_V173L', 'TP53_V173M', 'TP53_V216M', 'T